In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/datasets_for_colab_temp/urop/* /content

In [35]:
!git clone --recursive https://github.com/Microsoft/LightGBM
!cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 27815, done.
remote: Counting objects: 100% (3270/3270), done.
remote: Compressing objects: 100% (470/470), done.
remote: Total 27815 (delta 3015), reused 2926 (delta 2789), pack-reused 24545
Receiving objects: 100% (27815/27815), 19.69 MiB | 33.33 MiB/s, done.
Resolving deltas: 100% (20611/20611), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21733, done.        
remote: Counting objects: 100% (5/5), done.        
remot

In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def get_df_log_of(df_log, dname):
    df_log = df_log.copy()
    if dname == 'CV':
        df_log = df_log[(df_log['t_dat'] < '2019-10-02')]
        df_log['target'] = 'train'
        df_log.loc[(df_log['t_dat'] >= '2019-09-18'), 'target'] = 'valid'
        df_log.loc[(df_log['t_dat'] >= '2019-09-25'), 'target'] = 'test'
    elif dname == 'LB':
        df_log = df_log[(df_log['t_dat'] >= '2019-09-19')]
        df_log['target'] = 'train'
        df_log.loc[(df_log['t_dat'] >= '2020-09-16'), 'target'] = 'valid'
    else:
        raise Exception("dname = CV or LB only")
    return df_log

In [3]:
df_sub_raw = pd.read_parquet('raw/sample_submission.pq')

In [4]:
df_user = pd.read_parquet('data/df_user_preprocessed.pq')
df_log_all = pd.read_parquet('data/df_log_preprocessed.pq')
df_log_CV = get_df_log_of(df_log_all, 'CV')
df_log_LB = get_df_log_of(df_log_all, 'LB')

del df_log_all

In [ ]:
[1]*10**11

In [ ]:
df_user.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 1371980 entries, 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657 to ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   FN          1371980 non-null  int32  
 1   Active      1371980 non-null  int32  
 2   age         1371980 non-null  float64
 3   ACTIVE      1371980 non-null  uint8  
 4   LEFT CLUB   1371980 non-null  uint8  
 5   PRE-CREATE  1371980 non-null  uint8  
 6   Monthly     1371980 non-null  uint8  
 7   NONE        1371980 non-null  uint8  
 8   Regularly   1371980 non-null  uint8  
 9   age_gmm_1   1371980 non-null  float64
 10  age_gmm_2   1371980 non-null  float64
dtypes: float64(3), int32(2), uint8(6)
memory usage: 208.0 MB


In [5]:
df_user.reset_index(inplace=True)
df_user.drop(columns=['Active', 'age'], inplace=True)

df_user.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   customer_id  1371980 non-null  object 
 1   FN           1371980 non-null  int32  
 2   ACTIVE       1371980 non-null  uint8  
 3   LEFT CLUB    1371980 non-null  uint8  
 4   PRE-CREATE   1371980 non-null  uint8  
 5   Monthly      1371980 non-null  uint8  
 6   NONE         1371980 non-null  uint8  
 7   Regularly    1371980 non-null  uint8  
 8   age_gmm_1    1371980 non-null  float64
 9   age_gmm_2    1371980 non-null  float64
dtypes: float64(2), int32(1), object(1), uint8(6)
memory usage: 192.3 MB


In [6]:
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

In [7]:
df_log_CV['customer_id'] = customer_hex_id_to_int(df_log_CV['customer_id'])
df_log_LB['customer_id'] = customer_hex_id_to_int(df_log_LB['customer_id'])
df_log_CV['article_id'] = article_id_str_to_int(df_log_CV['article_id'])
df_log_LB['article_id'] = article_id_str_to_int(df_log_LB['article_id'])
df_log_CV.drop(columns='t_dat', inplace=True)
df_log_LB.drop(columns='t_dat', inplace=True)
df_user['customer_id'] = customer_hex_id_to_int(df_user['customer_id'])

In [8]:
test_week = df_log_CV['week'].iloc[-1]
c2weeks = df_log_CV.groupby('customer_id')['week'].unique()

c2weeks2shiftedweeks = {}
for id, weeks in c2weeks.items():
    c2weeks2shiftedweeks[id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shiftedweeks[id][weeks[i]] = weeks[i+1]
    c2weeks2shiftedweeks[id][weeks[-1]] = test_week

candidates_lastpurchase = df_log_CV.copy()

weeks = []
for id, week in zip(df_log_CV['customer_id'], df_log_CV['week']):
    weeks.append(c2weeks2shiftedweeks[id][week])
candidates_lastpurchase.week = weeks

In [9]:
del c2weeks
del c2weeks2shiftedweeks

In [10]:
mean_price = df_log_CV.groupby(['week', 'article_id'])['price'].mean()

sales = df_log_CV.groupby('week')['article_id'].value_counts()\
    .groupby('week').rank(method='dense', ascending=False)\
    .groupby('week').head(12).rename('bestseller_rank')

bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

unique_transactions = df_log_CV.groupby(['week', 'customer_id'])\
    .head(1).drop(columns=['article_id', 'price']).copy()
candidates_bestsellers = pd.merge(unique_transactions, bestsellers_previous_week, on='week')

test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

candidates_bestsellers_test_week = pd.merge(test_set_transactions, bestsellers_previous_week, on='week')

candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [11]:
del mean_price
del sales
del unique_transactions
del test_set_transactions
del candidates_bestsellers_test_week

In [12]:
import gc

gc.collect()

44

???

In [13]:
try:
    os.mkdir('data/temp')
except FileExistsError:
    print("folder already exists")

folder already exists


In [17]:
def mergewithfile(onepath, two, on, how, shrinksize=False):
    onelist = pd.read_csv(onepath, chunksize=200000)
    it = 0
    for o in onelist:
        if it==0:
            pd.merge(o, two, on=on, how=how).to_hdf('data/temp/tempdata.h5', 's', format='table', mode='w')
        else:
            if shrinksize:
                if np.random.randint(10)==0:
                    pd.merge(o, two, on=on, how=how).to_hdf('data/temp/tempdata.h5', 's', format='table', append=True)
            else:
                pd.merge(o, two, on=on, how=how).to_hdf('data/temp/tempdata.h5', 's', format='table', append=True)
        gc.collect()
        it += 1
        print(it, end=' ')
    del onelist
    gc.collect()

In [18]:
def mergewithfile2(onepath, two, on, how, shrinksize=False):
    onelist = pd.read_hdf(onepath, 's', chunksize=200000)
    it = 0
    for o in onelist:
        if it==0:
            pd.merge(o, two, on=on, how=how).to_hdf('data/temp/tempdata2.h5', 's', format='table', mode='w')
        else:
            if shrinksize:
                if np.random.randint(3)==0:
                    pd.merge(o, two, on=on, how=how).to_hdf('data/temp/tempdata2.h5', 's', format='table', append=True)
            else:
                pd.merge(o, two, on=on, how=how).to_hdf('data/temp/tempdata2.h5', 's', format='table', append=True)
        gc.collect()
        it += 1
        print(it, end=' ')
    del onelist
    gc.collect()

In [16]:
df_log_CV['purchased'] = 1

data = pd.concat([df_log_CV, candidates_lastpurchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)
gc.collect()

data.to_csv('data/temp/tempdata.csv')
del data
gc.collect()
data = mergewithfile('data/temp/tempdata.csv', bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
                     on=['week', 'article_id'], how='left', shrinksize=True)
del data
gc.collect()
data = pd.read_hdf('data/temp/tempdata.h5', 's')
gc.collect()
data = data[data.week != data.week.min()].copy()
gc.collect()
data.bestseller_rank.fillna(999, inplace=True)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [21]:
data.drop('Unnamed: 0', axis=1, inplace=True)
data['customer_id'] = data['customer_id'].astype('int64')

data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10600000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column            Dtype  
---  ------            -----  
 0   customer_id       int64  
 1   article_id        int64  
 2   price             float64
 3   sales_channel_id  int64  
 4   timestamp         int64  
 5   dto               object 
 6   week              int64  
 7   log_price         float64
 8   target            object 
 9   purchased         float64
 10  bestseller_rank   float64
dtypes: float64(4), int64(5), object(2)
memory usage: 2.1 GB


In [28]:
# store = pd.HDFStore('data/temp/tempdata3.h5')
# store.append('s', data, data_columns=True)
# store.close()
# del data
# gc.collect()
data = mergewithfile2('data/temp/tempdata3.h5', df_user, on='customer_id', how='left')
del data
gc.collect()
data = pd.read_hdf('data/temp/tempdata2.h5', 's')
gc.collect()

data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 

### below code results in memory error

In [ ]:
def mergebyparts(one, two, on, how, n=20):
    splits = np.array_split(one, n)
    ret = []
    for s in splits:
        ret.append(pd.merge(s, two, on=on, how=how))
    return pd.concat(ret)

In [ ]:
df_log_CV['purchased'] = 1

data = pd.concat([df_log_CV, candidates_lastpurchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

data = mergebyparts(data, bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'], how='left')
data = data[data.week != data.week.min()].copy()
data.bestseller_rank.fillna(999, inplace=True)

data = mergebyparts(data, df_item, on='article_id', how='left')

data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

### storing data in files instead

In [ ]:
def partmergetofile(splits, two, on, how, prefix):
    for i in range(20):
        pd.merge(splits[i], two, on=on, how=how).to_csv(f'data/temp/{prefix}{i}_temp.csv', index=False)
        gc.collect()

In [ ]:
df_log_CV['purchased'] = 1

data = pd.concat([df_log_CV, candidates_lastpurchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

splits = np.array_split(data, 20)
del data
gc.collect()
partmergetofile(splits, bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'], how='left', prefix='bestseller')

# combine files outside notebook here

In [ ]:
data = pd.read_csv('data/temp/bestseller_combined.csv')
data = data[data.week != data.week.min()].copy()
data.bestseller_rank.fillna(999, inplace=True)

splits = np.array_split(data, 20)
del data
gc.collect()
partmergetofile(splits, df_user_CV, on='customer_id', how='left', prefix='userdata')

# combine files outside notebook here

In [ ]:
data = pd.read_csv('data/temp/userdata_combined.csv')

data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [ ]:
data

In [ ]:
train = data[data.target != 'test']
test = data[data.target == 'test'].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

### training

In [29]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id'])

train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [30]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8530264 entries, 0 to 8530263
Data columns (total 20 columns):
 #   Column            Dtype  
---  ------            -----  
 0   customer_id       int64  
 1   article_id        int64  
 2   price             float64
 3   sales_channel_id  int64  
 4   timestamp         int64  
 5   dto               object 
 6   week              int64  
 7   log_price         float64
 8   target            object 
 9   purchased         float64
 10  bestseller_rank   float64
 11  FN                int32  
 12  ACTIVE            uint8  
 13  LEFT CLUB         uint8  
 14  PRE-CREATE        uint8  
 15  Monthly           uint8  
 16  NONE              uint8  
 17  Regularly         uint8  
 18  age_gmm_1         float64
 19  age_gmm_2         float64
dtypes: float64(6), int32(1), int64(5), object(2), uint8(6)
memory usage: 992.5+ MB


In [33]:
columns_to_use = ['article_id', 'sales_channel_id', 'timestamp',
                  'log_price', 'bestseller_rank', 'FN',
                  'ACTIVE', 'LEFT CLUB', 'PRE-CREATE',
                  'Monthly', 'NONE', 'Regularly',
                  'age_gmm_1', 'age_gmm_2']

In [34]:
train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

lightgbm

In [36]:
from lightgbm.sklearn import LGBMRanker

In [37]:
ranker = LGBMRanker(
    objective='lambdarank',
    metric='ndcg',
    boosting_type='goss',
    n_estimators=20,
    max_depth=3,
    importance_type='gain',
    verbose=10,
    device='gpu'
)

In [38]:
ranker = ranker.fit(
    train_X, train_y,
    group=train_baskets
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.983140
[LightGBM] [Info] Total Bins 841
[LightGBM] [Info] Number of data points in the train set: 8530264, number of used features: 14
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 9 dense feature groups (97.62 MB) transferred to GPU in 0.100154 secs. 1 sparse feature groups
[LightGBM] [Info] Using GOSS
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with leaves = 8 and depth = 3
[LightGBM] [Debug] Trained a tree with lea

In [39]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.5121878273178316
timestamp 0.4860835421185678
log_price 0.0015691273949578336
article_id 0.0001585900771431276
ACTIVE 2.959669285796765e-07
Regularly 2.930451988022991e-07
age_gmm_2 1.8845553614158337e-07
age_gmm_1 1.3562383607753934e-07
NONE 0.0
Monthly 0.0
PRE-CREATE 0.0
LEFT CLUB 0.0
FN 0.0
sales_channel_id 0.0


In [40]:
test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = ['0' + str(a_id) for a_id in bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()]

In [41]:
preds = []
for c_id in customer_hex_id_to_int(df_sub_raw.customer_id):
    if c_id in c_id2predicted_article_ids:
        ps = ['0' + str(p) for p in c_id2predicted_article_ids[c_id]]
        preds.append(ps)
    else:
        preds.append(bestsellers_last_week)

In [42]:
preds = [' '.join(ps) for ps in preds]
df_sub_raw.prediction = preds

In [43]:
try:
    os.mkdir('data/subs')
except FileExistsError:
    print("folder already exists")

In [44]:
df_sub_raw.to_csv('data/subs/lgbm_basic.csv.gz', index=False)